In [1]:
import pandas as pd
from selenium import webdriver
import time
import tqdm
from bs4 import BeautifulSoup as bs
import numpy as np

In [2]:
ENV_PATH = 'C:\\Users\\apply\\Desktop\\OneDrive - National University of Singapore\\industry-classification-master\\'

In [3]:
def get_BE(stock):
    url = "http://f10.eastmoney.com/NewFinanceAnalysis/Index?type=web&code=" + stock
    pat = "report_zcfzb"
    driver = webdriver.Chrome()
    page = driver.get(url)
    time.sleep(3)

    driver.find_element_by_xpath(
        "//*[@id='zcfzb_ul']/li/span[contains(text(),'按年度')]"
    ).click()
    time.sleep(3)
    
    content = driver.page_source.encode("utf-8")

    driver.close()

    soup = bs(content, "lxml")

    tab = soup.find("table", id=pat)

    fs_df = pd.read_html(str(tab), displayed_only=False)[0]
    

    return fs_df

In [4]:
def get_outstanding(stock):
    url = "http://f10.eastmoney.com/CapitalStockStructure/Index?type=web&code=" + stock
    pat = "lngbbd_Table"
    driver = webdriver.Chrome()
    page = driver.get(url)
    time.sleep(3)

    content = driver.page_source.encode("utf-8")

    soup = bs(content, "lxml")

    tab = soup.find("table", id=pat)

    fs_df = pd.read_html(str(tab), displayed_only=False)[0]
    
    driver.close()

    return fs_df

In [5]:
def calculat_BE(data):
    data.set_index('资产负债表',inplace=True)
    data = data.applymap(transfer_unit)
    try:
        perfer = data.loc['优先股  其中:优先股',:]
    except KeyError:
        perfer =  data.loc['优先股',:]
    try:
        bond = data.loc['永续债',:]
    except KeyError:
        bond =  data.loc['永续债  其中:永续债',:]
    
    equity = data.loc['归属于母公司股东权益合计',:]
    tax = data.loc['递延所得税负债',:]
    
    if not isinstance(perfer,pd.Series):
        perfer = perfer.sum()
    if not isinstance(bond,pd.Series):
        bond = bond.sum()
    if not isinstance(equity,pd.Series):
        equity = equity.sum()
    if not isinstance(tax,pd.Series):
        tax = tax.sum()
 
    BE = equity + tax - perfer - bond
    BE = pd.DataFrame(BE).T
    BE.index = [stock]
    return BE

In [6]:
def get_stock_outstanding(data):
    data.set_index('单位:万股',inplace=True)
    total = data.loc[['总股本','已上市流通A股'],:]
    
    col = data.columns.values.tolist()
    column = [i for i in col if 'Unnamed' not in i]
    column = column + [1]*(len(col)-len(column))
    total.columns = column
    total.index = pd.MultiIndex.from_product([[stock],['总股本','已上市流通A股']])
    
    return total
    

In [7]:
def transfer_unit(x):
    if isinstance(x,float):
        return x
    try:
        unit = ''.join([i for i in x[-2:] if not i.isdigit()])
    except:
        return 0
    if unit == '亿':
        x = float(x[:-len(unit)])*100000
    elif unit == '万亿':
        x = float(x[:-len(unit)])*100000000
    elif unit == '千万':
        x = float(x[:-len(unit)])*10000
    elif unit == '百万':
        x = float(x[:-len(unit)])*1000
    elif unit == '十万':
        x = float(x[:-len(unit)])*100
    elif unit == '万':
        x = float(x[:-len(unit)])*10
    else:
        x = 0
    return x

In [7]:
price = pd.read_csv(ENV_PATH+'data\\china share book value.csv')
price['date'] = pd.to_datetime(price['date'])
price = price[['symbol','date','adjusted']]

In [8]:
price = pd.read_csv(ENV_PATH+'data\\china share book value.csv')
stocks = price.iloc[:,0].values.tolist()
stocks

['SH603363',
 'SH603128',
 'SH688039',
 'SH603127',
 'SH601068',
 'SH601969',
 'SH603637',
 'SH603298',
 'SH603180',
 'SH688015',
 'SH603603',
 'SH603007',
 'SH600996',
 'SH603722',
 'SH603001',
 'SH603955',
 'SH688022',
 'SH600936',
 'SH603716',
 'SH603659',
 'SH603017',
 'SH603045',
 'SH601258',
 'SH601368',
 'SH688139',
 'SH601766',
 'SH603199',
 'SH688037',
 'SH603790',
 'SH603236',
 'SH601633',
 'SH603995',
 'SH601989',
 'SH601200',
 'SH601801',
 'SH603078',
 'SH601579',
 'SH601216',
 'SH603533',
 'SH600939',
 'SH601618',
 'SH600919',
 'SH601339',
 'SH603035',
 'SH603680',
 'SH603301',
 'SH601231',
 'SH688033',
 'SH688258',
 'SH603330',
 'SH603578',
 'SH603386',
 'SH603500',
 'SH603896',
 'SH603399',
 'SH603661',
 'SH601669',
 'SH601860',
 'SH603566',
 'SH601330',
 'SH601069',
 'SH603617',
 'SH603116',
 'SH603730',
 'SH603583',
 'SH603878',
 'SH603985',
 'SH688016',
 'SH601636',
 'SH603991',
 'SH601882',
 'SH603029',
 'SH603055',
 'SH601127',
 'SH603638',
 'SH603010',
 'SH603738',

In [8]:
stocks = price['symbol'].unique().tolist()
def transfer_ticker(x):
    if x[0]=='6':
        return 'SH'+x[:-3]
    elif x[0] =='0' or x[0]=='3':
        return 'SS'+x[:-3]
stocks = list(map(transfer_ticker,stocks))

In [12]:
BE = pd.DataFrame(columns = ['2019-12-31','2018-12-31','2017-12-31','2016-12-31','2015-12-31'])
for stock in tqdm.tqdm_notebook(stocks,desc='Companies fetched',unit='company'):
    try:
        BE_i = get_BE(stock)
        BE_i = calculat_BE(BE_i)
        BE = pd.concat([BE,BE_i])
    except:
        continue

d:\users\apply\anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [10]:
import tqdm
outstanding = pd.DataFrame(columns = ['2019-12-31'])
BE = pd.DataFrame(columns = ['2019-12-31','2018-12-31','2017-12-31','2016-12-31','2015-12-31'])
for stock in tqdm.tqdm_notebook(stocks,desc='Companies fetched',unit='company'):
    try:
        BE_i = get_BE(stock)
        outstanding_i = get_outstanding(stock)
        BE_i = calculat_BE(BE_i)
        outstanding_i = get_stock_outstanding(outstanding_i)
        outstanding_i = outstanding_i.loc[:,~outstanding_i.columns.duplicated()]
        outstanding = pd.concat([outstanding,outstanding_i])
        BE = pd.concat([BE,BE_i])
    except:
        continue


In [13]:
BE.to_csv(ENV_PATH+'data\\china share book value.csv')

In [11]:
outstanding.to_csv(ENV_PATH+'data\\china share outstanding.csv')
BE.to_csv(ENV_PATH+'data\\china share book value.csv')